In [1]:
# determing the packet length for each rank
import math
import os
import numpy as np
num_rank = 96
def get_packetLength_per_rank(num_rank): # This is the number of elements
    directory = 'GCN_matrices'
    dict_packet_length = {}
    dict_ratio = {}
    for filenm in os.listdir(directory): # iterate over all files
        file_name = os.path.join(directory, filenm)
        if os.path.isfile(file_name): # if it is a file
            splitted_file_name = file_name.split("/")[1].split("_")
            dataset = splitted_file_name[0]
            print(file_name)
            bucket_list = [0 for g in range(num_rank)] # total: both diagonal and off-diagonal
            bucket_list_diagonal = [0 for g in range(num_rank)] # only diagonal
            with open(file_name) as f:
                line1 = f.readline() 
                line2 = int(f.readline())
                lines = f.readlines()
                num_row = int(line1.split()[0])
                num_col = int(line1.split()[1])
                bucket_length = int(num_row/num_rank)
                for i in range(line2):
#                     print(lines[i])
#                     print(i)
#                     print(row)
                    row = int(lines[i].split()[0])
                    col = int(lines[i].split()[1])
                    bucket = int(row/bucket_length)
                    if bucket > num_rank - 1: # if we are out of range as a result of rounding bucket_length \\
                        # just add these outliers to the last rank
                        bucket = num_rank - 1
                    bucket_list[bucket] += 1
                    if col >= bucket*bucket_length and col < (bucket +1) * bucket_length:
                        bucket_list_diagonal[bucket] += 1
            ratio_diag = np.array(bucket_list_diagonal)/np.array(bucket_list)
            # ratio of diag over total (both diagonal and off-diagonal)
            if splitted_file_name[1] == 'A.txt': # A matrix is being used for both layers
                key1 = dataset + '_L1_AXW'
                key2 = dataset + '_L2_AXW'
                dict_packet_length[key1] = bucket_list
                dict_packet_length[key2] = bucket_list
                dict_ratio[key1] = ratio_diag
                dict_ratio[key2] = ratio_diag
            elif splitted_file_name[1] == 'feat': 
                if splitted_file_name[2] == 'L1.txt':
                    key = dataset + '_L1_XW'
                    dict_packet_length[key] = bucket_list
                    dict_ratio[key] = ratio_diag
                elif splitted_file_name[2] == 'L2.txt':
                    key = dataset + '_L2_XW'
                    dict_packet_length[key] = bucket_list
                    dict_ratio[key] = ratio_diag
                else:
                    print('unsupported file name')
            else:
                print('unsupported file name')
    return dict_packet_length, dict_ratio

In [2]:
dict_packet_length, ratio_diag = get_packetLength_per_rank(24)
for key in dict_packet_length:
    print(key)
    print('packet length', np.mean(np.array(dict_packet_length[key])))
    print('ratio diag', np.mean(np.array(ratio_diag[key])))
# print(dict_packet_length)
# print('-------------------------------------------')
# print(ratio_diag)

GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/PPI_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/PPI_feat_L2.txt
GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
Cora_L2_XW
packet length 1524.0833333333333
ratio diag 0.041666666666666664
Cora_L1_AXW
packet length 439.8333333333333
ratio diag 0.2718319319435809
Cora_L2_AXW
packet length 439.8333333333333
ratio diag 0.2718319319435809
Cora_L1_XW
packet length 2050.6666666666665
ratio diag 0.042571262815470745
PPI_L1_XW
packet length 1177.9583333333333
ratio diag 0.041666666666666664
Pubmed_L1_AXW
packet length 3693.7916666666665
ratio diag 0.1080134416875822

In [2]:
# Simulating a switch
from constant import *
import math
import random as ra


def sweep(sweep_threshold, sweep_step, payload_length, num_port):
    # payload_length is a list and is in Bytes
    ra.seed(1)
    results = []
    time_stamp = [0 for j in range(num_port)]
    sweep_num = int(sweep_threshold/sweep_step)
    num_expr = sweep_num ** num_port
    verbose_freq = 100 # showing the progress for each segment
    verbose_segment = int(num_expr/verbose_freq)
    verbose_num = 0
    for i in range(num_expr):
        # print(i)
        if i%verbose_segment == 0:
            print(verbose_num, " percent")
            verbose_num += 1
        # print(time_stamp)
        delay = comp_delay(time_stamp, payload_length, num_port, 'default', 0)
        results.append(delay)
        # print(delay)
        time_stamp[0] += sweep_step
        for k in range(num_port):
            if time_stamp[k] == sweep_threshold:
                time_stamp[k] = 0
                if k != num_port - 1:
                    time_stamp[k+1] += sweep_step
    return results

def sweepRand(num_expr, sweep_threshold, sweep_step, payload_length, num_port):
    # payload_length is a list and is in Bytes
    ra.seed(1)
    results = []
    sweep_num = int(sweep_threshold/sweep_step)
    verbose_freq = 100 # showing the progress for each segment
    verbose_segment = int(num_expr/verbose_freq)
    verbose_num = 0
    for i in range(num_expr):
        # print(i)
        if i%verbose_segment == 0:
            print(verbose_num, " percent")
            verbose_num += 1
        # print(time_stamp)
        # time_stamp[0] += sweep_step
        time_stamp = [ra.randrange(0, sweep_threshold, sweep_step) for j in range(num_port)]
        delay = comp_delay(time_stamp, payload_length, num_port, 'default', [0 for j in range(num_port)])
        results.append(delay)
        # print(delay)
    print('finished')
    return results

def comp_delay_wrapper(time_stamp, payload_length, num_port, op_type, inp_packet_delay, fpga_mode):
    if fpga_mode == 'FaaS':
        switch_lat = SWITCH_LATENCY_NETFPGA
    elif fpga_mode == 'FatS':
        switch_lat = SWITCH_LATENCY_OMNIPATH + UDP_ALVEO_LATENCY
    else: 
        print ("Incorrect fpga_mode")
    mytime = 0
    end_time_port = [0 for i in range(num_port)]
    curr_queue = 0 # pointer to which queue
#     print('Hey: num port', num_port)
    num_beats = [math.ceil((payload_length[k] + HEADER_LENGTH)/PHIT_SIZE) for k in range(num_port)]
    if op_type == 'default':
        packet_delay = [num_beats[k] * PERIOD_CLK for k in range(num_port)]
    else:
        packet_delay = inp_packet_delay
    queuefinished = [False for i in range(num_port)]
    # num_finished = 0
    
    while all(queuefinished) == False:
        if mytime >= time_stamp[curr_queue] and queuefinished[curr_queue] == False: #schedule an consume the packet
            queuefinished[curr_queue] = True
            mytime += packet_delay[curr_queue]
            end_time_port[curr_queue] = mytime + switch_lat
            # print("queue id ", curr_queue, " is finished", "num_queue finished is: ", num_finished, "out of: ", num_port)
            # num_finished += 1
        
        # round-robin
        if (curr_queue == num_port - 1):
            curr_queue = 0
        else:
            curr_queue += 1
        
        # one clock delay for checking the next queue
        mytime += PERIOD_CLK 
    
    end_time_global = mytime + switch_lat
    return end_time_global, end_time_port

def comp_delay(time_stamp, payload_length, num_port, op_type, inp_packet_delay, fpga_mode):
    # time is considered as ns
    # time_stamp is a list
    # payload_length is a list and is in Bytes
    # inp_packet_delay is a list
    
    if num_port < NUM_PORTS_MAX:
        num_port_leaf = num_port 
        num_leaf_switch = 1
    else:
        num_port_leaf = NUM_PORTS_MAX
        num_leaf_switch = int(num_port/NUM_PORTS_MAX)
    end_time_global_wrapper = [0 for k in range(num_leaf_switch)]
    end_time_port_wrapper = [0 for k in range(num_port)]
#     print('Hey: num_leaf_switch', num_leaf_switch)
    
    for i in range(num_leaf_switch):
        end_time_global_wrapper[i], end_time_port_wrapper[i*num_port_leaf: (i+1)*num_port_leaf] = comp_delay_wrapper(
                           time_stamp[i*num_port_leaf: (i+1)*num_port_leaf], 
                           payload_length[i*num_port_leaf: (i+1)*num_port_leaf], 
                           num_port_leaf,
                           op_type,
                           inp_packet_delay[i*num_port_leaf: (i+1)*num_port_leaf],
                           fpga_mode)
    end_time_global = max(end_time_global_wrapper)
    
    return end_time_global, end_time_port_wrapper
               
def analysis(results):
    # print(results)
    print('max: ', max(results))
    print('min: ', min(results))

In [9]:
# fpga results
######## UNCOMMENT OS IF YOU WANT TO DO OUTPUT STATIONARY ############
from constant import *
def instruction_count(instr, vlen):
    if instr == 'vse32_v':
        return vlen + 6
    elif instr == 'vle32_v':
        return vlen + 6
    elif instr == 'vmacc_xv': # dense
        return vlen + PE_latency
    elif instr == 'vmacc_vx': # dense
        return vlen + PE_latency
    elif instr == 'spvmacc_xv': # sparse
        #     return vlen + PE_latency # lets factor out vlen and lump it with all off-diag nnz and instead only measure the overhead
        return 1 + PE_LATENCY # 1 is b/c of packet_length at the begining of packet
    elif instr == 'spvmacc_vx': # spearse
#         return vlen + PE_LATENCY # lets factor out vlen and lump it with all off-diag nnz and instead only measure the overhead
        return 1 + PE_LATENCY # 1 is b/c of packet_length at the begining of packet   
    elif instr == 'vstreamout_v':
        return vlen * NUM_COL
    elif instr == 'vsetivli':
        return 1
    elif instr == 'wfi':
        return 1
    elif (instr == 'addi' or instr == 'lui' or instr == 'add' or instr == 'bne'): # scalar
        return 1
    else:
        print('unsupported instruction')

def gcn_FPGA_time_cgra(instruction, packet_length, SIMD_under_utilization):
    num_rank = len(packet_length)
    cgra_time = [0 for i in range(num_rank)]
    for i in range(num_rank):
        if packet_length[i] == 0:
            cgra_time[i] = 0 # we should bypass cgra
        else:
            time_instr = 0 # init to zero
            for key in instruction:
                instr_vlen = instruction[key][0]
                instr_cnt = instruction[key][1] # instruction count
                instr_name = key.split('-')[0]
                time_instr += (instruction_count(instr_name, instr_vlen))*instr_cnt
            cgra_time[i] = (int(packet_length[i]/(SIMD_DEGREE*SIMD_under_utilization)) + time_instr) *PERIOD_CLK #*1e-9 # in ns
    return cgra_time

def gcn_FPGA_time_cgra_simulator(d, total_rank): # more accurate (from simulator)
    overhead_loading_time = 440 # ns
    num_rep_column_PPI = 4 # b/c it has 121 columns and we only have 31 columns in LA
    
    ########. UNCOMMENT THIS IF YOU WANT TO DO OUTPUT STATIONARY ############
    
    ### xV (OS, output satationary)
#     if d == 'Cora':
#         # rank: time
#         F_time = {2:68164.02, 4:35875.8, 8:18882, 16:12322, 24:12056}
#     elif d == 'Citeseer':
#         F_time = {2:83889.18, 4:44152.2, 8:23238, 16:15188, 24:14884}
#     elif d == 'Pubmed':
# #         F_time = {2: 824076.36, 4: 433724.4, 8: 228276, 16: 148902, 24: 145620} # old results
#         F_time = {2: 1647854, 4: 867292, 8: 432872, 16: 259104, 24: 172220, 48: 85336}
# #     elif d == 'Reddit':
# #         F_time = {2: 217337003.2, 4: 114387896.4, 8: 60204156, 16: 30359191, 24:20582278}
#     elif d == 'PPI':
#         F_time = {2: 59700*num_rep_column_PPI, 4: 30020*num_rep_column_PPI, 8: 15180*num_rep_column_PPI, 16: 7672*num_rep_column_PPI, 24: 7624*num_rep_column_PPI}
#     elif d == 'ogbnproducts':
#         F_time = {2: 358944716, 4: 179032828, 8: 89076884, 16: 44098912, 24: 22056632}
        
    #### Vx: (WS, weight stationary)
    if d=='Cora':
        F_time={2:54767, 4:52014, 8:50440, 16:48913, 24:47432}
    elif d=='Citeseer':
        F_time ={2:69166, 4:63767, 8:62785, 16:61803, 24:57345}
    elif d=='Pubmed':
        F_time={2:510881, 4:507653, 8:430009, 16:391188, 24:373270}
    elif d=='PPI':
        F_time = {2:37212*num_rep_column_PPI, 4:31276*num_rep_column_PPI, 8:28308*num_rep_column_PPI, 16:26824*num_rep_column_PPI, 24:26188*num_rep_column_PPI}
    elif d=='ogbnproducts':
        F_time = {2: 664450533, 4: 369435093, 8: 221927373, 16: 148173513, 24: 74419653}
        
    cgra_time = [F_time[total_rank] for i in range(total_rank)]
    return cgra_time

In [10]:
# main func for getting results
def get_results(d,total_rank, num_iter, TM, TK, vmacc_unroll_factor, vmacc_vlen, SIMD_under_utilization, fpga_comm, fpga_mode):
    instruction = {'addi-0':[0, 5],
                   'lui-0':[0, 1],
                   'wfi-0':[0, 1],
                   # outer loop
                   'vsetivli-0':[0, 2*TM],
                   'vle32_v-0':[vmacc_unroll_factor, TM],
                   'addi-1':[0, 2*TM],
                   'bne-0':[0, TM],
                   'vstreamout_v-0':[vmacc_unroll_factor, TM],
                   # inner loop
                   'vsetivli-1':[0, 2*TM*TK],
                   'vle32_v-1':[vmacc_vlen, TM*TK],
                   'add-0':[0, TM*TK],
                   'addi-2':[0, TM*TK],
                   'bne-1':[0, TM*TK],
                   'spvmacc_xv-0':[vmacc_vlen, TM*TK*vmacc_unroll_factor]}
    
    # This is the raw data, storing all of the ranks
    my_dict_cpu_unroll = {d + '_L1_XW': [[] for i in range(3 * num_iter)],
                          d + '_L1_AXW': [[] for i in range(3 * num_iter)],
                          d + '_L2_XW': [[] for i in range(3 * num_iter)],
                          d + '_L2_AXW': [[] for i in range(3 * num_iter)]}
    
    # This is the actual cpu result (max of all ranks)
    my_dict_cpu = {d + '_L1_XW': [0 for i in range(num_iter)],
                   d + '_L1_AXW': [0 for i in range(num_iter)],
                   d + '_L2_XW': [0 for i in range(num_iter)],
                   d + '_L2_AXW': [0 for i in range(num_iter)]}
   

    my_dict_fpga = {d + '_L1_AXW':[0 for i in range(num_iter)],
                    d + '_L2_AXW':[0 for i in range(num_iter)]}

    DIRECTORY = './GCN_results/new_results/' + str(total_rank) + '/'
    
    cpu_breakdown_comp = {d + '_L1_XW': [0 for i in range(num_iter)],
                   d + '_L1_AXW': [0 for i in range(num_iter)],
                   d + '_L2_XW': [0 for i in range(num_iter)],
                   d + '_L2_AXW': [0 for i in range(num_iter)]}
    
    cpu_breakdown_comm = {d + '_L1_XW': [0 for i in range(num_iter)],
                   d + '_L1_AXW': [0 for i in range(num_iter)],
                   d + '_L2_XW': [0 for i in range(num_iter)],
                   d + '_L2_AXW': [0 for i in range(num_iter)]}
    
    for filenm in os.listdir(DIRECTORY):
        split = filenm.split('_')
        dataset = split[1]
        if dataset != d:
            continue
        layer = split[2]
        iteration = int(split[3])
        tot_rank = int(split[4].split('.')[0])
        if (tot_rank != total_rank):
            continue
        file_name = os.path.join(DIRECTORY, filenm)
#         print(DIRECTORY, d, tot_rank, total_rank)
        
        if iteration != 1:
            with open(file_name) as f:
                f.readline()
                f.readline()
                lines = f.readlines()
                for i in range(tot_rank): #xW
                    split_lines = lines[i].split(" ")
                    rank = int(split_lines[0])
                    tot_time = float(split_lines[2])
#                     comm_time = float(split_lines[3])
                    comp_time = float(split_lines[4].strip())
                    key = dataset +  '_'  + layer + '_' + 'XW'
                    my_dict_cpu_unroll[key][(iteration - 2)].append(tot_time) # -2 b/c first valid iter is offset by 2
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter].append(0.0)
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter*2].append(comp_time)
                    
                for i in range(tot_rank): #A(xW)
                    j = i + tot_rank
                    split_lines = lines[j].split(" ")
                    rank = int(split_lines[0])
                    tot_time = float(split_lines[2])
                    comm_time = float(split_lines[3])
                    comp_time = float(split_lines[4].strip())
                    key = dataset + '_' + layer + '_' + 'AXW' 
                    my_dict_cpu_unroll[key][(iteration-2)].append(tot_time) # -2 b/c the first valid iteration is 2
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter].append(comm_time)
                    my_dict_cpu_unroll[key][(iteration-2) + num_iter*2].append(comp_time)
        
        
    for key in my_dict_cpu_unroll: # take the maximum among the ranks
        result_list = my_dict_cpu_unroll[key]
        max_idx = [0 for i in range(num_iter)]
        for j in range(3 * num_iter):
            if j < num_iter:
                max_idx[j] = (np.argmax(result_list[j]))
            elif j >= num_iter and j < 2 * num_iter:
                cpu_breakdown_comm[key][j% num_iter] = result_list[j][max_idx[j%num_iter]] * 1000.0
            elif j >= 2 * num_iter and j < 3 * num_iter:
                cpu_breakdown_comp[key][j% num_iter] = result_list[j][max_idx[j%num_iter]] * 1000.0

    fpga_breakdown_comp = {d + '_L1_AXW':[0 for i in range(num_iter)],
                    d + '_L2_AXW':[0 for i in range(num_iter)]}
    fpga_breakdown_comm = {d + '_L1_AXW':[0 for i in range(num_iter)],
                    d + '_L2_AXW':[0 for i in range(num_iter)]}
    # Lets calculate my_dict_cpu and my_dict_fpga
    dict_packet_length, ratio_diag = get_packetLength_per_rank(total_rank) # This is the number of elements
#     print(DIRECTORY, d, tot_rank, total_rank)
    for key in my_dict_cpu_unroll:
        result_list = my_dict_cpu_unroll[key]
        time_stamp = [[0 for h in range(total_rank)] for g in range(num_iter)]
        for j in range(3*num_iter):
#             if key[-3:] == 'AXW': # otherwise if its XW we will not accelerate it
            if key[-3:] == 'AXW': # otherwise if its XW we will not accelerate it
                if j < num_iter: # tot time calculate max(tot_time)
                    max_result = max(result_list[j])
                    my_dict_cpu[key][j%num_iter] = max_result * 1000.0 # in ms
                elif j >= num_iter and j < 2*num_iter: # comm (we get the timestamp from here)
                    max_result = max(result_list[j])
                    time_stamp[j%num_iter] = max_result - np.array(result_list[j]) # save time_stamp for later
                elif j >= 2*num_iter and j < 3*num_iter: # comp
                    this_ratio_diag = np.array(ratio_diag[key])
                    this_packet_length = np.array(dict_packet_length[key])
                    this_result_list = np.array(result_list[j])
                    result_comp_eff = this_result_list * this_ratio_diag # only diagonal
                    max_result = max(result_comp_eff * 1000.0) # in ms 
                    min_result = min(result_comp_eff * 1000.0) # in ms
                    packet_length = this_packet_length * (1.0 - this_ratio_diag) # only off-diagonal # This is the number of elements
    #                 time_stamp = np.array(result_list[j]) - min_result
#                     gcn_fpga_time = gcn_FPGA_time_cgra(instruction, packet_length, SIMD_under_utilization)# depends on the rank b/c different ranks have different packet length
                    gcn_fpga_time = gcn_FPGA_time_cgra_simulator(d, total_rank)# depends on the rank b/c different ranks have different packet length
                    
                    # gcn_fpga_time is a list but switch_time_global is a single time
                    switch_time_global, switch_time_port = comp_delay(time_stamp[j%num_iter], packet_length, total_rank, 'gcn', gcn_fpga_time, fpga_mode)
#                     print("my FPGA", d, total_rank, switch_time_port)
                    tot_fpga_time = max((result_comp_eff * 1000.0) + (np.array(switch_time_port) * 1e-6)) # in ms
                    # breakdown
                    max_idx = np.argmax((result_comp_eff * 1000.0) + (np.array(switch_time_port) * 1e-6))
                    fpga_breakdown_comp[key][j % num_iter] = result_comp_eff[max_idx] * 1000.0 # in ms
                    fpga_breakdown_comm[key][j % num_iter] = np.array(switch_time_port)[max_idx] * 1e-6 # in ms
                    
                    # 1000 and 1e-6 are for converting sec to ms and ns to ms respectively
                    my_dict_fpga[key][j%num_iter] = tot_fpga_time # ms
    #                 if verbose == True:
    #                     print('max comp fpga: ', max_result)
    #                     print('min comp fpga: ', min_result)
            else: # XW
                if j < num_iter: # tot time calculate max(tot_time)
                    max_result = max(result_list[j])
                    my_dict_cpu[key][j%num_iter] = max_result * 1000.0 # in ms
                
    print('--------- final result -----------')
    print('cpu results: ', my_dict_cpu)
    print('cpu-comp: ', cpu_breakdown_comp)
    print('cpu-comm: ', cpu_breakdown_comm)
    print('fpga results: ', my_dict_fpga)
    print('fpga-comp: ', fpga_breakdown_comp)
    print('fpga-comm: ', fpga_breakdown_comm)
    print('----------------------------------')
    return my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm

In [11]:
total_ranks = [2, 4, 8, 16, 24]
# total_ranks = [2]
# total_ranks_Reddit = [2, 4, 8, 16, 24]
TM = {'Cora': [1], 'Citeseer': [1], 'Pubmed' : [3,3,1,1,1], 'PPI' : [3,3,1,1,1], 'Reddit': [40,19,10,5,3], 'ogbnproducts': [3,3,1,1,1]} 
TK = {'Cora': 2, 'Citeseer': 3, 'Pubmed' : 10, 'PPI':10, 'Reddit': 155, 'ogbnproducts': 10}
vmacc_unroll_factor = {'Cora': [14,7,4,2,1], 'Citeseer': [17,9,5,2,1], 'Pubmed' : [32,16,25,13,7], 'PPI' : [32,16,25,13,7], 'Reddit': [32,32,32,32,32], 'ogbnproducts': [32,16,25,13,7]}
vmacc_vlen = {'Cora': 1354, 'Citeseer': 1109, 'Pubmed' : 2048, 'PPI' : 2048, 'Reddit': 1503, 'ogbnproducts': 2048} 
SIMD_under_utilization = {'Cora': [1,1,14/16,15/16,13/16], 'Citeseer': [1,15/16, 14/16, 1,1], 'Pubmed' : [1,1,1,1,15/16], 'PPI' : [1,1,1,1,15/16], 'Reddit': [1,1,15/16,15/16,13/16], 'ogbnproducts': [1,1,1,1,1]}
num_iter = 5
# CHANGE THIS IF YOU WANNA HAVE ANOTHER ACCELRATION MODE (OPTIONS ARE FaaS and FatS)
this_fpga_mode = 'FaaS'

# This is the result for socket programming using sfml
# fpga_comm = {'Cora': [0.001036, 0.001, 0.001075, 0.000934, 0.000754],
#              'Citeseer': [0.001159, 0.001194, 0.001003, 0.000986, 0.000833],
#              'Pubmed': [0.00456, 0.002228, 0.001422, 0.00116, 0.000784],
#              'Reddit': [0.005332, 0.02666, 0.02211, 0.006873, 0.004554, 0.004554, 0.004554]}

# This is the result for MPI based on exact message size
fpga_comm = {'Cora': [0.000025, 0.000004, 0.000004, 0.000003, 0.000004],
             'Citeseer': [0.000003, 0.000004, 0.000009, 0.000004, 0.000003],
             'Pubmed': [0.000005, 0.000004, 0.000004, 0.000002, 0.000003],
#              'Reddit': [0.000090, 0.000090, 0.000090, 0.000090, 0.000090],
             'PPI': [0.000025, 0.000004, 0.000004, 0.000003, 0.000004],
            'ogbnproducts': [0.000005, 0.000004, 0.000004, 0.000002, 0.000003]}

num_packetization = {2: 3498, 4: 1749, 8: 875, 16:438, 24:292} # for reddit
map_rank = {2:0, 4:1, 8:2, 16:3, 24:4}

In [12]:
# MAIN FUNC FOR REPORTING RESULTS
datasets = ['Cora', 'Citeseer', 'Pubmed', 'PPI', 'ogbnproducts']
d_result = [[],[],[],[],[]]
for d in datasets:
    if d == 'Cora':
        for (rank,vuf,simd) in zip(total_ranks, vmacc_unroll_factor['Cora'], SIMD_under_utilization['Cora']):
            print('Cora >> Rank ' + str(rank) + ' Num Iter '\
                    + str(num_iter) + ' TM ' + str(TM['Cora'][0]) + ' VUF ' + str(vuf) + ' VmV ' + str(vmacc_vlen['Cora']) + ' SIMD ' + str(simd))
            my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm = get_results(d,rank, num_iter, TM['Cora'][0], TK['Cora'], vuf, vmacc_vlen['Cora'], simd, fpga_comm['Cora'], this_fpga_mode)
            d_result[0].append([my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm])
    if d == 'Citeseer':
        for (rank,vuf,simd) in  zip(total_ranks, vmacc_unroll_factor['Citeseer'], SIMD_under_utilization['Citeseer']):
            print('Citeseer >> Rank ' + str(rank) + ' Num Iter '\
                    + str(num_iter) + ' TM ' + str(TM['Citeseer'][0]) + ' VUF ' + str(vuf) + ' VmV ' + str(vmacc_vlen['Citeseer']) + ' SIMD ' + str(simd))
            my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm = get_results(d,rank, num_iter, TM['Citeseer'][0], TK['Citeseer'], vuf, vmacc_vlen['Citeseer'], simd, fpga_comm['Citeseer'], this_fpga_mode)  
            d_result[1].append([my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm])

    if d == 'Pubmed':
        for (rank, tm, vuf,simd) in  zip(total_ranks, TM['Pubmed'], vmacc_unroll_factor['Pubmed'], SIMD_under_utilization['Pubmed']):
            print('Pubmed >> Rank ' + str(rank) + ' Num Iter '\
                    + str(num_iter) + ' TM ' + str(tm) + ' VUF ' + str(vuf) + ' VmV ' + str(vmacc_vlen['Pubmed']) + ' SIMD ' + str(simd))
            my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm = get_results(d,rank, num_iter, tm, TK['Pubmed'], vuf, vmacc_vlen['Pubmed'], simd, fpga_comm['Pubmed'], this_fpga_mode)
            d_result[2].append([my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm])
    
    if d == 'PPI':
        for (rank, tm, vuf,simd) in  zip(total_ranks, TM['PPI'], vmacc_unroll_factor['PPI'], SIMD_under_utilization['PPI']):
            print('PPI >> Rank ' + str(rank) + ' Num Iter '\
                    + str(num_iter) + ' TM ' + str(tm) + ' VUF ' + str(vuf) + ' VmV ' + str(vmacc_vlen['PPI']) + ' SIMD ' + str(simd))
            my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm = get_results(d,rank, num_iter, tm, TK['PPI'], vuf, vmacc_vlen['PPI'], simd, fpga_comm['PPI'], this_fpga_mode)
            d_result[3].append([my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm])    
    if d == 'ogbnproducts':
        for (rank, tm, vuf,simd) in  zip(total_ranks, TM['ogbnproducts'], vmacc_unroll_factor['ogbnproducts'], SIMD_under_utilization['ogbnproducts']):
            print('ogbnproducts >> Rank ' + str(rank) + ' Num Iter '\
                    + str(num_iter) + ' TM ' + str(tm) + ' VUF ' + str(vuf) + ' VmV ' + str(vmacc_vlen['ogbnproducts']) + ' SIMD ' + str(simd))
            my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm = get_results(d,rank, num_iter, tm, TK['ogbnproducts'], vuf, vmacc_vlen['ogbnproducts'], simd, fpga_comm['ogbnproducts'], this_fpga_mode)
            d_result[4].append([my_dict_cpu, cpu_breakdown_comp, cpu_breakdown_comm, my_dict_fpga, fpga_breakdown_comp, fpga_breakdown_comm])


Cora >> Rank 2 Num Iter 5 TM 1 VUF 14 VmV 1354 SIMD 1
GCN_matrices/Cora_feat_L2.txt
GCN_matrices/Cora_A.txt
GCN_matrices/Cora_feat_L1.txt
GCN_matrices/PPI_feat_L1.txt
GCN_matrices/Pubmed_A.txt
GCN_matrices/PPI_feat_L2.txt
GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
--------- final result -----------
cpu results:  {'Cora_L1_XW': [0.611901, 0.646448, 0.6138870000000001, 0.6471950000000001, 0.619637], 'Cora_L1_AXW': [0.260866, 0.266822, 0.255243, 0.270887, 0.259055], 'Cora_L2_XW': [0.211292, 0.210344, 0.213363, 0.210573, 0.224713], 'Cora_L2_AXW': [0.145192, 0.158437, 0.143221, 0.13524000000000003, 0.14234100000000002]}
cpu-comp:  {'Cora_L1_XW': [0.5031770000

GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
--------- final result -----------
cpu results:  {'Cora_L1_XW': [0.338904, 0.347144, 0.352773, 0.351071, 0.32367], 'Cora_L1_AXW': [0.276721, 0.29175799999999996, 0.301908, 0.296021, 0.27520700000000003], 'Cora_L2_XW': [0.112536, 0.106743, 0.092823, 0.114882, 0.100993], 'Cora_L2_AXW': [0.209046, 0.198277, 0.241588, 0.242537, 0.198408]}
cpu-comp:  {'Cora_L1_XW': [0.043301, 0.039765999999999996, 0.04638, 0.039868, 0.046424999999999994], 'Cora_L1_AXW': [0.007703000000000001, 0.008721, 0.010384, 0.010428000000000002, 0.005097], 'Cora_L2_XW': [0.024949000000000002, 0.024377, 0.024857, 0.028982, 0.015201000000000001], 

GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
--------- final result -----------
cpu results:  {'Citeseer_L1_XW': [0.865957, 0.868974, 0.905543, 0.9000250000000001, 0.855258], 'Citeseer_L1_AXW': [0.229776, 0.224881, 0.231065, 0.253594, 0.238327], 'Citeseer_L2_XW': [0.09477200000000001, 0.089753, 0.088325, 0.096719, 0.093113], 'Citeseer_L2_AXW': [0.15476600000000001, 0.166161, 0.14066099999999998, 0.163112, 0.232032]}
cpu-comp:  {'Citeseer_L1_XW': [0.142409, 0.147792, 0.133464, 0.148086, 0.14269900000000002], 'Citeseer_L1_AXW': [0.013517, 0.013454, 0.028050000000000002, 0.013441999999999999, 0.023294000000000002], 'Citeseer_L2_XW': [0.020946, 0.0199969999999

GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
--------- final result -----------
cpu results:  {'Pubmed_L1_XW': [2.83032, 2.75027, 2.80367, 2.80885, 2.7456300000000002], 'Pubmed_L1_AXW': [0.946546, 0.92399, 0.961822, 0.9564419999999999, 0.967081], 'Pubmed_L2_XW': [0.21762600000000001, 0.21746800000000002, 0.21437699999999998, 0.214486, 0.220697], 'Pubmed_L2_AXW': [0.39202600000000004, 0.369245, 0.364595, 0.375103, 0.36149499999999996]}
cpu-comp:  {'Pubmed_L1_XW': [2.74146, 2.65907, 2.72423, 2.6645499999999998, 2.65868], 'Pubmed_L1_AXW': [0.486827, 0.47497700000000004, 0.48528200000000005, 0.487871, 0.48828299999999997], 'Pubmed_L2_XW': [0.15270999999999998,

GCN_matrices/PPI_feat_L2.txt
GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
--------- final result -----------
cpu results:  {'PPI_L1_XW': [0.08388100000000001, 0.08137899999999999, 0.086781, 0.081485, 0.089899], 'PPI_L1_AXW': [0.315737, 0.323657, 0.31502199999999997, 0.31867799999999996, 0.309067], 'PPI_L2_XW': [0.22822399999999998, 0.226555, 0.214719, 0.220548, 0.217917], 'PPI_L2_AXW': [2.0017, 2.02981, 2.00532, 1.9934200000000002, 1.9825200000000003]}
cpu-comp:  {'PPI_L1_XW': [0.029071999999999997, 0.026666, 0.028555999999999998, 0.027023, 0.016399], 'PPI_L1_AXW': [0.210291, 0.214002, 0.21334899999999998, 0.21316, 0.213924], 'PPI_L2_XW': [0.145303, 0.1633

GCN_matrices/PPI_feat_L2.txt
GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
--------- final result -----------
cpu results:  {'ogbnproducts_L1_XW': [2208.9500000000003, 2210.2400000000002, 2332.92, 2207.03, 2332.75], 'ogbnproducts_L1_AXW': [41.675200000000004, 42.459900000000005, 42.116099999999996, 42.3021, 41.9527], 'ogbnproducts_L2_XW': [353.67900000000003, 354.05199999999996, 354.524, 356.354, 364.75800000000004], 'ogbnproducts_L2_AXW': [129.447, 129.206, 128.81, 128.992, 129.679]}
cpu-comp:  {'ogbnproducts_L1_XW': [2208.8199999999997, 2210.12, 2332.8, 2206.9, 2332.62], 'ogbnproducts_L1_AXW': [19.7841, 20.4618, 19.8263, 20.4488, 19.959999999999997], 'ogb

GCN_matrices/PPI_feat_L2.txt
GCN_matrices/ogbnproducts_A.txt
GCN_matrices/Reddit_A.txt
GCN_matrices/Citeseer_feat_L2.txt
GCN_matrices/Citeseer_feat_L1.txt
GCN_matrices/PPI_A.txt
GCN_matrices/ogbnproducts_feat_L1.txt
GCN_matrices/ogbnproducts_feat_L2.txt
GCN_matrices/Citeseer_A.txt
GCN_matrices/Pubmed_feat_L2.txt
GCN_matrices/Reddit_feat_L2.txt
GCN_matrices/Pubmed_feat_L1.txt
GCN_matrices/Reddit_feat_L1.txt
--------- final result -----------
cpu results:  {'ogbnproducts_L1_XW': [200.986, 243.989, 201.16, 200.74, 228.488], 'ogbnproducts_L1_AXW': [37.1761, 38.146699999999996, 32.620000000000005, 36.921099999999996, 36.669399999999996], 'ogbnproducts_L2_XW': [31.500500000000002, 32.8353, 34.266999999999996, 33.7431, 34.115], 'ogbnproducts_L2_AXW': [85.4815, 92.79419999999999, 87.601, 83.34020000000001, 87.59110000000001]}
cpu-comp:  {'ogbnproducts_L1_XW': [200.747, 243.752, 200.942, 200.549, 228.191], 'ogbnproducts_L1_AXW': [2.49198, 2.83841, 2.4811400000000003, 2.5235499999999997, 2.82767

In [14]:
import matplotlib.pyplot as plt
for idx, d in enumerate(d_result): #dictionary
    if idx == 0: #cora
        for trial, (rank,vuf,simd) in enumerate(zip(total_ranks, vmacc_unroll_factor['Cora'], SIMD_under_utilization['Cora'])):
            if rank == 144:
                print('Cora >> Rank: '+ str(rank), '|| vmacc_unroll_factor ' + str(vuf), '|| SIMD ' + str(simd))
                print('==========CPU comp ============')
                print('L1 XW: ',d[trial][1]['Cora_L1_XW']) #cpu comp
                print('L1 AXW: ', d[trial][1]['Cora_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Cora_L2_XW'])
                print('L2 AXW: ', d[trial][1]['Cora_L2_AXW'])
                print('==========CPU comm ============')
                print('L1 XW: ',d[trial][2]['Cora_L1_XW']) #cpu comm
                print('L1 AXW: ', d[trial][2]['Cora_L1_AXW']) 
                print('L2 XW: ', d[trial][2]['Cora_L2_XW'])
                print('L2 AXW: ', d[trial][2]['Cora_L2_AXW'])
                print('==========FPGA comp ============')
                print('L1 XW: ',d[trial][1]['Cora_L1_XW']) #fpga comp
                print('L1 AXW: ', d[trial][4]['Cora_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Cora_L2_XW'])
                print('L2 AXW: ', d[trial][4]['Cora_L2_AXW'])
                print('==========FPGA comm ============')
                print('L1 XW: ',d[trial][2]['Cora_L1_XW']) #fpga comm
                print('L1 AXW: ', np.array(d[trial][5]['Cora_L1_AXW']) + np.array([fpga_comm['Cora'][map_rank[rank]] for i in range(num_iter)])) 
                print('L2 XW: ', d[trial][2]['Cora_L2_XW'])
                print('L2 AXW: ', np.array(d[trial][5]['Cora_L2_AXW']) + np.array([fpga_comm['Cora'][map_rank[rank]] for i in range(num_iter)]))
                print('=========================')

                print()
                print('CPU AVG: ', np.mean(np.array(d[trial][0]['Cora_L1_XW']) + np.array(d[trial][0]['Cora_L1_AXW']) + np.array(d[trial][0]['Cora_L2_XW']) + np.array(d[trial][0]['Cora_L2_AXW'])))
                print('FPGA AVG: ', np.mean(np.array(d[trial][0]['Cora_L1_XW']) + np.array(d[trial][3]['Cora_L1_AXW']) + np.array(fpga_comm['Cora']) + np.array(d[trial][0]['Cora_L2_XW']) + np.array(d[trial][3]['Cora_L2_AXW']) + np.array([fpga_comm['Cora'][map_rank[rank]] for i in range(num_iter)])))
                print('CPU STD: ', np.std(np.array(d[trial][0]['Cora_L1_XW']) + np.array(d[trial][0]['Cora_L1_AXW']) + np.array(d[trial][0]['Cora_L2_XW']) + np.array(d[trial][0]['Cora_L2_AXW'])))
                print('FPGA STD: ', np.std(np.array(d[trial][0]['Cora_L1_XW']) + np.array(d[trial][3]['Cora_L1_AXW']) + np.array(fpga_comm['Cora']) + np.array(d[trial][0]['Cora_L2_XW']) + np.array(d[trial][3]['Cora_L2_AXW']) + np.array([fpga_comm['Cora'][map_rank[rank]] for i in range(num_iter)])))
                print()
        print()
        print()
        print()
    if idx == 1: #citeseer
        for trial, (rank,vuf,simd) in enumerate(zip(total_ranks, vmacc_unroll_factor['Citeseer'], SIMD_under_utilization['Citeseer'])):
            if rank == 144:
                print('Citeseer >> Rank: '+ str(rank), '|| vmacc_unroll_factor ' + str(vuf), '|| SIMD ' + str(simd))
                print('==========CPU comp ============')
                print('L1 XW: ',d[trial][1]['Citeseer_L1_XW']) #cpu comp
                print('L1 AXW: ', d[trial][1]['Citeseer_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Citeseer_L2_XW'])
                print('L2 AXW: ', d[trial][1]['Citeseer_L2_AXW'])
                print('==========CPU comm ============')
                print('L1 XW: ',d[trial][2]['Citeseer_L1_XW']) #cpu comm
                print('L1 AXW: ', d[trial][2]['Citeseer_L1_AXW']) 
                print('L2 XW: ', d[trial][2]['Citeseer_L2_XW'])
                print('L2 AXW: ', d[trial][2]['Citeseer_L2_AXW'])
                print('==========FPGA comp ============')
                print('L1 XW: ',d[trial][1]['Citeseer_L1_XW']) #fpga comp
                print('L1 AXW: ', d[trial][4]['Citeseer_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Citeseer_L2_XW'])
                print('L2 AXW: ', d[trial][4]['Citeseer_L2_AXW'])
                print('==========FPGA comm ============')
                print('L1 XW: ',d[trial][2]['Citeseer_L1_XW']) #fpga comm
                print('L1 AXW: ', np.array(d[trial][5]['Citeseer_L1_AXW']) + np.array([fpga_comm['Citeseer'][map_rank[rank]] for i in range(num_iter)]))
                print('L2 XW: ', d[trial][2]['Citeseer_L2_XW'])
                print('L2 AXW: ', np.array(d[trial][5]['Citeseer_L2_AXW']) + np.array([fpga_comm['Citeseer'][map_rank[rank]] for i in range(num_iter)]))
                print('=========================')

                print()
                print('CPU AVG: ', np.mean(np.array(d[trial][0]['Citeseer_L1_XW']) + np.array(d[trial][0]['Citeseer_L1_AXW']) + np.array(d[trial][0]['Citeseer_L2_XW']) + np.array(d[trial][0]['Citeseer_L2_AXW'])))
                print('FPGA AVG: ', np.mean(np.array(d[trial][0]['Citeseer_L1_XW']) + np.array(d[trial][3]['Citeseer_L1_AXW']) + np.array(fpga_comm['Citeseer']) + np.array(d[trial][0]['Citeseer_L2_XW']) + np.array([fpga_comm['Citeseer'][map_rank[rank]] for i in range(num_iter)])))
                print('CPU STD: ', np.std(np.array(d[trial][0]['Citeseer_L1_XW']) + np.array(d[trial][0]['Citeseer_L1_AXW']) + np.array(d[trial][0]['Citeseer_L2_XW']) + np.array(d[trial][0]['Citeseer_L2_AXW'])))
                print('FPGA STD: ', np.std(np.array(d[trial][0]['Citeseer_L1_XW']) + np.array(d[trial][3]['Citeseer_L1_AXW']) + np.array(fpga_comm['Citeseer']) + np.array(d[trial][0]['Citeseer_L2_XW']) + np.array([fpga_comm['Citeseer'][map_rank[rank]] for i in range(num_iter)])))
                print()
        print()
        print()
        print()
    if idx == 2: # pubmed
        for trial, (rank,vuf,simd) in enumerate(zip(total_ranks, vmacc_unroll_factor['Pubmed'], SIMD_under_utilization['Pubmed'])):
            if rank == 144:
                print('Pubmed >> Rank: '+ str(rank), '|| vmacc_unroll_factor ' + str(vuf), '|| SIMD ' + str(simd))
                print('==========CPU comp ============')
                print('L1 XW: ',d[trial][1]['Pubmed_L1_XW']) #cpu comp
                print('L1 AXW: ', d[trial][1]['Pubmed_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Pubmed_L2_XW'])
                print('L2 AXW: ', d[trial][1]['Pubmed_L2_AXW'])
                print('==========CPU comm ============')
                print('L1 XW: ',d[trial][2]['Pubmed_L1_XW']) #cpu comm
                print('L1 AXW: ', d[trial][2]['Pubmed_L1_AXW']) 
                print('L2 XW: ', d[trial][2]['Pubmed_L2_XW'])
                print('L2 AXW: ', d[trial][2]['Pubmed_L2_AXW'])
                print('==========FPGA comp ============')
                print('L1 XW: ',d[trial][1]['Pubmed_L1_XW']) #fpga comp
                print('L1 AXW: ', d[trial][4]['Pubmed_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Pubmed_L2_XW'])
                print('L2 AXW: ', d[trial][4]['Pubmed_L2_AXW'])
                print('==========FPGA comm ============')
                print('L1 XW: ',d[trial][2]['Pubmed_L1_XW']) #fpga comm
                print('L1 AXW: ', np.array(d[trial][5]['Pubmed_L1_AXW']) + np.array([fpga_comm['Pubmed'][map_rank[rank]] for i in range(num_iter)]))
                print('L2 XW: ', d[trial][2]['Pubmed_L2_XW'])
                print('L2 AXW: ', np.array(d[trial][5]['Pubmed_L2_AXW']) + np.array([fpga_comm['Pubmed'][map_rank[rank]] for i in range(num_iter)]))
                print('=========================')

                print()
                print('CPU AVG: ', np.mean(np.array(d[trial][0]['Pubmed_L1_XW']) + np.array(d[trial][0]['Pubmed_L1_AXW']) + np.array(d[trial][0]['Pubmed_L2_XW']) + np.array(d[trial][0]['Pubmed_L2_AXW'])))
                print('FPGA AVG: ', np.mean(np.array(d[trial][0]['Pubmed_L1_XW']) + np.array(d[trial][3]['Pubmed_L1_AXW']) + np.array(fpga_comm['Pubmed']) + np.array(d[trial][0]['Pubmed_L2_XW']) + np.array(d[trial][3]['Pubmed_L2_AXW']) + np.array([fpga_comm['Pubmed'][map_rank[rank]] for i in range(num_iter)])))
                print('CPU STD: ', np.std(np.array(d[trial][0]['Pubmed_L1_XW']) + np.array(d[trial][0]['Pubmed_L1_AXW']) + np.array(d[trial][0]['Pubmed_L2_XW']) + np.array(d[trial][0]['Pubmed_L2_AXW'])))
                print('FPGA STD: ', np.std(np.array(d[trial][0]['Pubmed_L1_XW']) + np.array(d[trial][3]['Pubmed_L1_AXW']) + np.array(fpga_comm['Pubmed']) + np.array(d[trial][0]['Pubmed_L2_XW']) + np.array(d[trial][3]['Pubmed_L2_AXW']) + np.array([fpga_comm['Pubmed'][map_rank[rank]] for i in range(num_iter)])))
                print()
        print()
        print()
        print()

    if idx == 3: # reddit
        for trial, (rank,vuf,simd) in enumerate(zip(total_ranks_Reddit, vmacc_unroll_factor['Reddit'], SIMD_under_utilization['Reddit'])):
            if rank == 144:
                print('Reddit >> Rank: '+ str(rank), '|| vmacc_unroll_factor ' + str(vuf), '|| SIMD ' + str(simd))
                print('==========CPU comp ============')
                print('L1 XW: ',d[trial][1]['Reddit_L1_XW']) #cpu comp
                print('L1 AXW: ', d[trial][1]['Reddit_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Reddit_L2_XW'])
                print('L2 AXW: ', d[trial][1]['Reddit_L2_AXW'])
                print('==========CPU comm ============')
                print('L1 XW: ',d[trial][2]['Reddit_L1_XW']) #cpu comm
                print('L1 AXW: ', d[trial][2]['Reddit_L1_AXW']) 
                print('L2 XW: ', d[trial][2]['Reddit_L2_XW'])
                print('L2 AXW: ', d[trial][2]['Reddit_L2_AXW'])
                print('==========FPGA comp ============')
                print('L1 XW: ',d[trial][1]['Reddit_L1_XW']) #fpga comp
                print('L1 AXW: ', d[trial][4]['Reddit_L1_AXW']) 
                print('L2 XW: ', d[trial][1]['Reddit_L2_XW'])
                print('L2 AXW: ', d[trial][4]['Reddit_L2_AXW'])
                print('==========FPGA comm ============')
                print('L1 XW: ',d[trial][2]['Reddit_L1_XW']) #fpga comm
                print('L1 AXW: ', np.array(d[trial][5]['Reddit_L1_AXW']) + np.array([fpga_comm['Reddit'][map_rank[rank]] for i in range(num_iter)]))
                print('L2 XW: ', d[trial][2]['Reddit_L2_XW'])
                print('L2 AXW: ', np.array(d[trial][5]['Reddit_L2_AXW']) + np.array([fpga_comm['Reddit'][map_rank[rank]] for i in range(num_iter)]))
                print('=========================')

                print()
                print('CPU AVG: ', np.mean(np.array(d[trial][0]['Reddit_L1_XW']) + np.array(d[trial][0]['Reddit_L1_AXW']) + np.array(d[trial][0]['Reddit_L2_XW']) + np.array(d[trial][0]['Reddit_L2_AXW'])))
                print('FPGA AVG: ', np.mean(np.array(d[trial][0]['Reddit_L1_XW']) + np.array(d[trial][3]['Reddit_L1_AXW']) + np.array(fpga_comm['Reddit']) + np.array(d[trial][0]['Reddit_L2_XW']) + np.array(d[trial][3]['Reddit_L2_AXW']) + np.array([fpga_comm['Reddit'][map_rank[rank]] for i in range(num_iter)])))
                print('CPU STD: ', np.std(np.array(d[trial][0]['Reddit_L1_XW']) + np.array(d[trial][0]['Reddit_L1_AXW']) + np.array(d[trial][0]['Reddit_L2_XW']) + np.array(d[trial][0]['Reddit_L2_AXW'])))
                print('FPGA STD: ', np.std(np.array(d[trial][0]['Reddit_L1_XW']) + np.array(d[trial][3]['Reddit_L1_AXW']) + np.array(fpga_comm['Reddit']) + np.array(d[trial][0]['Reddit_L2_XW']) + np.array(d[trial][3]['Reddit_L2_AXW']) + np.array([fpga_comm['Reddit'][map_rank[rank]] for i in range(num_iter)])))
                print()
            

Cora >> Rank: 144 || vmacc_unroll_factor 1 || SIMD 0.8125
==========CPU comp ============
L1 XW:  [0.007862000000000001, 0.009619, 0.007487, 0.008583, 0.008022999999999999]
L1 AXW:  [0.001993, 0.001297, 0.0023120000000000003, 0.0018780000000000001, 0.0038299999999999996]
L2 XW:  [0.002903, 0.0029330000000000003, 0.002888, 0.0029579999999999997, 0.002927]
L2 AXW:  [0.001967, 0.001771, 0.001579, 0.0027140000000000003, 0.001593]
==========CPU comm ============
L1 XW:  [1.40328, 1.67177, 1.8113599999999999, 1.58321, 1.23517]
L1 AXW:  [0.6055, 0.612517, 0.955494, 0.586941, 0.579458]
L2 XW:  [0.654559, 0.883493, 0.723522, 1.10288, 1.1486999999999998]
L2 AXW:  [0.761464, 0.697051, 0.742101, 0.984806, 1.06441]
==========FPGA comp ============
L1 XW:  [0.007862000000000001, 0.009619, 0.007487, 0.008583, 0.008022999999999999]
L1 AXW:  [0.002681142857142857, 0.002614114285714286, 0.0028326857142857145, 0.002391657142857143, 0.002614114285714286]
L2 XW:  [0.002903, 0.0029330000000000003, 0.002888,

ValueError: operands could not be broadcast together with shapes (5,) (7,) 

In [43]:
### reduction
fpga_mode = 'FaaS'
ms_sweep = 8
num_packetization = [1, 1, 1, 1, 1, 1, 4, 16]
map_node = {48:2}
fpga_comm_mpi = 0.000090 #sec
collective_fpga_mess_time = {
#                              48: [1320, 1320, 4952, 14552, 52952, 206552, 823338, 3290732]
#                              48: [624, 624, 4256, 13856, 52256, 205856, 822642, 3290036]
                             48: [13, 13, 88.6666666, 288.6666667, 1088.666667, 4288.666667, 17138.375, 68542.41667],
                             576:[30.54861111, 30.54861111 ,80.72222222, 273.3888889]
} #ns
# my_ranks = [48, 192, 576]
my_ranks = [48, 576]
for num_rank in my_ranks:
    print('num_rank', num_rank)
    dummy = [10 for i in range(num_rank)]
    skew = [0 for i in range(num_rank)]
    for ms in range(ms_sweep):
        tot_fpga_time = 0.0
        collective_fpga_time = [collective_fpga_mess_time[num_rank][ms] for i in range(num_rank)]
        switch_time_global, switch_time_port = comp_delay(skew, dummy, num_rank, 'collective', collective_fpga_time, fpga_mode)
    #                     print("my FPGA", d, total_rank, switch_time_port)
        tot_fpga_time = max((np.array(switch_time_port) * 1e-6)) # in ms
#         print(tot_fpga_time)
        tot_fpga_time += fpga_comm_mpi*1000
#         print(fpga_comm_mpi*1000)
        tot_fpga_time += (num_packetization[ms]-1)*((fpga_comm_mpi*1000)+(SWITCH_LATENCY_NETFPGA*1e-6))   
#         print((num_packetization[ms]-1)*((fpga_comm_mpi*1000)+(SWITCH_LATENCY_NETFPGA*1e-6)))
        print(tot_fpga_time)
#         print('----------------')

num_rank 48
0.09098300000000001
0.09098300000000001
0.09461499999680001
0.10421500000160003
0.14261500001599997
0.2962150000160002
1.183514
4.732960000160002
num_rank 576
0.11006699999935995
0.11006699999935995
0.13896699999871992
0.24994300000639802


IndexError: list index out of range